## **Task 5: Mental Health Support Chatbot (Fine-Tuned)**

In [1]:
!pip install transformers datasets torch accelerate streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.

In [2]:
# Mental Health Support Chatbot - Fine-tuned LLM
# Complete implementation for Google Colab

# ============================================================================
# INSTALLATION AND SETUP
# ============================================================================

# Run this cell first to install required packages

!pip install huggingface_hub wandb

import os
import json
import torch
import pandas as pd
import numpy as np
from datetime import datetime
from typing import List, Dict, Any
import warnings
warnings.filterwarnings('ignore')

# Transformers and datasets
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    pipeline
)
from datasets import Dataset, load_dataset
import streamlit as st

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ============================================================================
# DATA PREPARATION AND PREPROCESSING
# ============================================================================

class EmpatheticDataProcessor:
    """Process EmpatheticDialogues dataset for mental health support training"""

    def __init__(self):
        self.supportive_prompts = [
            "I understand how you're feeling. ",
            "That sounds really difficult. ",
            "I hear you, and your feelings are valid. ",
            "It's completely normal to feel this way. ",
            "Thank you for sharing that with me. ",
            "I can sense that this is important to you. "
        ]

    def load_empathetic_dialogues(self):
        """Load and preprocess the EmpatheticDialogues dataset"""
        try:
            # Try loading the dataset with different methods
            print("Attempting to load EmpatheticDialogues dataset...")

            # Method 1: Try the standard approach
            try:
                dataset = load_dataset("empathetic_dialogues", trust_remote_code=True)
                print(f"✅ Successfully loaded dataset with {len(dataset['train'])} training samples")
                return dataset
            except:
                pass

            # Method 2: Try alternative loading
            try:
                dataset = load_dataset("facebook/empathetic_dialogues")
                print(f"✅ Successfully loaded dataset with {len(dataset['train'])} training samples")
                return dataset
            except:
                pass

            # If both fail, create enhanced sample dataset
            print("Using enhanced sample dataset for training...")
            return self.create_enhanced_sample_dataset()

        except Exception as e:
            print(f"Error loading dataset: {e}")
            return self.create_enhanced_sample_dataset()

    def create_enhanced_sample_dataset(self):
        """Create an enhanced sample dataset with more training examples"""
        sample_data = [
            # Anxiety-related conversations
            {
                "context": "I'm feeling really anxious about my job interview tomorrow.",
                "response": "I understand how nerve-wracking job interviews can be. It's completely normal to feel anxious about something important like this. Remember that they invited you because they saw potential in your application. Try some deep breathing exercises tonight, and remember that you've prepared well for this moment."
            },
            {
                "context": "I can't stop worrying about everything that could go wrong.",
                "response": "Worry can feel overwhelming when our minds focus on all the possible negative outcomes. It's like your brain is trying to protect you by preparing for every scenario. Sometimes it helps to ask yourself: 'What would I tell a good friend who was feeling this way?' Your worries are valid, but they don't define what will actually happen."
            },
            {
                "context": "My heart races and I feel like I can't breathe when I'm anxious.",
                "response": "Those physical symptoms of anxiety can be really frightening. When we're anxious, our body goes into fight-or-flight mode, which causes those racing heart and breathing changes. Try the 4-7-8 breathing technique: breathe in for 4, hold for 7, exhale for 8. This can help calm your nervous system."
            },

            # Depression-related conversations
            {
                "context": "I've been feeling so overwhelmed with everything lately.",
                "response": "That sounds really difficult, and I want you to know that feeling overwhelmed is something many people experience. It's okay to acknowledge these feelings. Sometimes breaking things down into smaller, manageable steps can help. What's one small thing you could focus on today?"
            },
            {
                "context": "I feel empty and nothing seems to matter anymore.",
                "response": "That emptiness you're describing sounds incredibly painful. When depression takes hold, it can make everything feel meaningless and gray. Please know that these feelings, while very real right now, don't reflect your true worth or the reality of your life. You matter, and there are people who care about you."
            },
            {
                "context": "I don't have energy to do anything, even simple tasks feel impossible.",
                "response": "Depression can make even the smallest tasks feel like climbing a mountain. That lack of energy isn't laziness - it's a real symptom of what you're going through. Be gentle with yourself. Maybe start with just one tiny thing, like drinking a glass of water or stepping outside for a moment. Small steps count too."
            },

            # Self-worth and confidence issues
            {
                "context": "I feel like I'm not good enough at anything.",
                "response": "I hear you, and those feelings of self-doubt can be really painful. Your worth isn't determined by your achievements or comparisons to others. Everyone has unique strengths and qualities. Can you think of one thing you've done recently that you're proud of, even if it seems small?"
            },
            {
                "context": "Everyone else seems to have their life together except me.",
                "response": "That comparison trap is so common, and social media often makes it worse by showing everyone's highlight reels. The truth is, most people are struggling with something, even if it's not visible. Your journey is unique, and it's okay to be figuring things out. Progress isn't always linear."
            },
            {
                "context": "I made a mistake at work and now I feel terrible about myself.",
                "response": "Making mistakes is part of being human, even though it can feel devastating in the moment. One mistake doesn't define your worth or your capabilities. What matters is how you learn from it and move forward. Can you show yourself the same compassion you'd show a friend in this situation?"
            },

            # Sleep and worry issues
            {
                "context": "I'm having trouble sleeping because I keep worrying.",
                "response": "Sleep difficulties due to worry are very common, and I understand how frustrating that cycle can be. Creating a calming bedtime routine might help - perhaps some gentle music, reading, or meditation. Writing down your worries before bed can sometimes help quiet your mind too."
            },
            {
                "context": "My mind won't stop racing when I try to sleep.",
                "response": "A racing mind at bedtime is exhausting. Your brain might be trying to process the day or prepare for tomorrow. Try the 'mental filing cabinet' technique: imagine putting each worry into a folder and filing it away to deal with tomorrow. Progressive muscle relaxation can also help shift your focus from thoughts to your body."
            },

            # Loneliness and isolation
            {
                "context": "I feel so lonely even when I'm around people.",
                "response": "That kind of loneliness can feel particularly isolating because it seems contradictory, but it's actually very common. Sometimes we can feel disconnected even in a crowd. Building genuine connections takes time, and it's okay to start small. Your feelings are valid, and reaching out like this shows strength."
            },
            {
                "context": "I don't think anyone really understands me.",
                "response": "Feeling misunderstood can be incredibly isolating. It's like being surrounded by people speaking a different language. The desire to be truly seen and understood is so human. Sometimes it helps to start by understanding ourselves better first, and then finding people who share similar experiences or values."
            },
            {
                "context": "All my friends seem too busy for me lately.",
                "response": "It's hard when it feels like the people we care about don't have time for us. Sometimes people get caught up in their own struggles and it's not personal, but that doesn't make it hurt less. Have you considered reaching out directly to one friend? Sometimes people don't realize we need connection."
            },

            # Stress and pressure
            {
                "context": "I'm stressed about my relationship ending.",
                "response": "The end of a relationship can be one of life's most challenging experiences, and it's completely understandable that you're feeling stressed. Grief over a relationship is real and valid. Take time to process your emotions, and remember that healing isn't linear. You deserve support during this difficult time."
            },
            {
                "context": "I have so much pressure from my family to succeed.",
                "response": "Family pressure can create such internal conflict - wanting to make them proud while also staying true to yourself. It's exhausting to feel like you're constantly being measured against expectations. Remember that your worth isn't tied to meeting everyone else's definition of success. What does success mean to you?"
            },
            {
                "context": "I'm struggling to balance work and personal life.",
                "response": "Work-life balance is challenging for so many people, especially when both areas demand so much from us. It's like trying to fill two buckets with one stream of water. Setting boundaries can feel scary but it's essential for your wellbeing. What's one small boundary you could set this week?"
            },

            # Past and future worries
            {
                "context": "I can't stop thinking about my mistakes from the past.",
                "response": "Ruminating over past mistakes is something many people struggle with, and it can be exhausting. While it's natural to reflect, dwelling on what we can't change often increases our suffering. Consider what you've learned from these experiences - that growth is valuable. Practicing self-compassion can help break this cycle."
            },
            {
                "context": "I'm scared about what the future holds.",
                "response": "Uncertainty about the future can trigger so much anxiety. Our brains want to predict and control what's coming, but the truth is that none of us really know what lies ahead. While we can't control everything, we can focus on building resilience and coping skills for whatever comes. What's one thing you can control today?"
            },

            # Motivation and purpose
            {
                "context": "I feel like giving up on my dreams.",
                "response": "Feeling like giving up can be a sign that you're carrying a heavy emotional load right now. Dreams and goals can feel overwhelming when we're struggling. It's okay to take a step back and reassess, but remember that setbacks don't mean failure. Sometimes the path to our dreams looks different than we initially imagined."
            },
            {
                "context": "I don't know what my purpose in life is.",
                "response": "Questions about purpose are some of the deepest we can ask, and it's completely normal not to have clear answers. Purpose often emerges through experience rather than revelation. What activities make you feel most like yourself? What problems do you naturally want to help solve? Purpose can be found in both big and small ways."
            }
        ]

        # Create train/validation split
        train_size = int(0.8 * len(sample_data))
        train_data = Dataset.from_list(sample_data[:train_size])
        val_data = Dataset.from_list(sample_data[train_size:])

        return {"train": train_data, "validation": val_data}

    def format_conversation(self, context: str, response: str) -> str:
        """Format conversation for training"""
        # Add supportive elements to make responses more empathetic
        formatted_response = response

        # Create training format
        conversation = f"<|user|> {context} <|assistant|> {formatted_response} <|endoftext|>"
        return conversation

    def prepare_training_data(self, dataset, tokenizer, max_length=512):
        """Prepare data for training"""
        def tokenize_function(examples):
            # Handle both single examples and batches
            if isinstance(examples['context'], str):
                # Single example
                text = self.format_conversation(examples['context'], examples['response'])
                texts = [text]
            else:
                # Batch of examples
                texts = [
                    self.format_conversation(context, response)
                    for context, response in zip(examples['context'], examples['response'])
                ]

            # Tokenize
            tokenized = tokenizer(
                texts,
                truncation=True,
                padding=True,
                max_length=max_length,
                return_tensors="pt"
            )

            # For causal LM, labels are the same as input_ids
            tokenized["labels"] = tokenized["input_ids"].clone()

            return tokenized

        # Apply tokenization
        tokenized_dataset = dataset.map(
            tokenize_function,
            batched=True,
            remove_columns=dataset.column_names
        )

        return tokenized_dataset

# ============================================================================
# MODEL TRAINING CLASS
# ============================================================================

class MentalHealthChatbotTrainer:
    """Fine-tune a language model for mental health support"""

    def __init__(self, model_name="microsoft/DialoGPT-small"):
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        self.trainer = None

    def setup_model_and_tokenizer(self):
        """Initialize model and tokenizer"""
        print(f"Loading model: {self.model_name}")

        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, padding_side='left')

        # Add special tokens if they don't exist
        special_tokens = {
            "pad_token": "<|pad|>",
            "eos_token": "<|endoftext|>",
            "additional_special_tokens": ["<|user|>", "<|assistant|>"]
        }

        num_added_tokens = self.tokenizer.add_special_tokens(special_tokens)

        # Load model
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
        )

        # Resize token embeddings if new tokens were added
        if num_added_tokens > 0:
            self.model.resize_token_embeddings(len(self.tokenizer))

        print(f"Model loaded. Vocabulary size: {len(self.tokenizer)}")

    def train_model(self, train_dataset, eval_dataset=None, output_dir="./mental-health-chatbot"):
        """Fine-tune the model"""

        # Training arguments - fixed for compatibility
        training_args = TrainingArguments(
            output_dir=output_dir,
            overwrite_output_dir=True,
            num_train_epochs=3,
            per_device_train_batch_size=2,
            per_device_eval_batch_size=2,
            gradient_accumulation_steps=4,
            warmup_steps=100,
            logging_dir=f"{output_dir}/logs",
            logging_steps=50,
            save_steps=500,
            eval_steps=500 if eval_dataset else None,
            eval_strategy="steps" if eval_dataset else "no",  # Fixed parameter name
            save_strategy="steps",
            load_best_model_at_end=True if eval_dataset else False,
            metric_for_best_model="eval_loss" if eval_dataset else None,
            greater_is_better=False,
            report_to=[],  # Disable wandb for simplicity
            fp16=torch.cuda.is_available(),
            dataloader_pin_memory=False,
            remove_unused_columns=False,
            push_to_hub=False,
            hub_model_id=None,
        )

        # Data collator
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer,
            mlm=False,  # We're doing causal LM, not masked LM
        )

        # Initialize trainer
        self.trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            data_collator=data_collator,
        )

        # Train the model
        print("Starting model training...")
        print(f"Training samples: {len(train_dataset)}")
        if eval_dataset:
            print(f"Validation samples: {len(eval_dataset)}")

        try:
            self.trainer.train()
            print("✅ Training completed successfully!")
        except Exception as e:
            print(f"❌ Training error: {e}")
            # Try with reduced parameters
            print("🔄 Trying with simplified training parameters...")

            # Simplified training arguments
            training_args = TrainingArguments(
                output_dir=output_dir,
                overwrite_output_dir=True,
                num_train_epochs=2,
                per_device_train_batch_size=1,
                gradient_accumulation_steps=2,
                logging_steps=10,
                save_steps=100,
                eval_strategy="no",
                save_strategy="epoch",
                report_to=[],
                dataloader_pin_memory=False,
                remove_unused_columns=False,
            )

            # Recreate trainer with simplified args
            self.trainer = Trainer(
                model=self.model,
                args=training_args,
                train_dataset=train_dataset,
                data_collator=data_collator,
            )

            # Try training again
            self.trainer.train()
            print("✅ Training completed with simplified parameters!")

        # Save the final model
        try:
            self.trainer.save_model()
            self.tokenizer.save_pretrained(output_dir)
            print(f"📁 Model saved to {output_dir}")
        except Exception as e:
            print(f"⚠️ Error saving model: {e}")

        print("🎉 Training pipeline completed!")

    def load_trained_model(self, model_path="./mental-health-chatbot"):
        """Load a trained model"""
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_path)
            self.model = AutoModelForCausalLM.from_pretrained(
                model_path,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
            )
            print(f"Trained model loaded from {model_path}")
            return True
        except Exception as e:
            print(f"Error loading trained model: {e}")
            return False

# ============================================================================
# CHATBOT INTERFACE CLASS
# ============================================================================

class MentalHealthChatbot:
    """Mental health support chatbot interface"""

    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.conversation_history = []

        # Supportive response templates
        self.response_templates = [
            "I understand that this must be difficult for you.",
            "Your feelings are completely valid.",
            "Thank you for sharing this with me.",
            "It takes courage to talk about these things.",
            "I'm here to listen and support you.",
        ]

    def generate_response(self, user_input: str, max_length=150, temperature=0.8) -> str:
        """Generate empathetic response to user input"""

        # Check for greeting or simple inputs first
        user_lower = user_input.lower().strip()

        # Handle greetings
        if any(word in user_lower for word in ['hi', 'hello', 'hey', 'good morning', 'good afternoon']):
            return "Hello! I'm glad you're here. I'm a supportive listener, and I'm here to help you through whatever you're experiencing. How are you feeling today?"

        # Handle "how are you" questions
        if any(phrase in user_lower for phrase in ['how are you', 'how do you feel', 'what about you']):
            return "Thank you for asking. I'm here and ready to focus entirely on you and what you're going through. What's on your mind today?"

        # Use keyword-based empathetic responses for common emotions
        emotion_responses = {
            'sad': "I can hear that you're feeling sad right now. Sadness can feel overwhelming sometimes. It's completely natural to have these feelings. Would you like to share what's been making you feel this way?",
            'anxious': "Anxiety can be really difficult to deal with. Your feelings are valid, and it takes courage to acknowledge them. What's been causing you to feel anxious lately?",
            'depressed': "I'm sorry you're going through such a difficult time. Depression can make everything feel heavy and overwhelming. You're not alone in this. Can you tell me more about what you've been experiencing?",
            'angry': "It sounds like you're feeling really frustrated or angry about something. Those are powerful emotions, and it's okay to feel them. What's been triggering these feelings for you?",
            'lonely': "Feeling lonely can be one of the most painful experiences. Even when we're surrounded by people, we can still feel isolated. I want you to know that you're not truly alone. Can you share more about what's making you feel this way?",
            'stressed': "Stress can really take a toll on both our minds and bodies. It sounds like you're carrying a heavy load right now. What are some of the main things that have been stressing you out?",
            'worried': "Worry can consume so much of our mental energy. It's understandable that you're feeling this way. Sometimes talking through our worries can help make them feel more manageable. What's been on your mind?",
            'scared': "Feeling scared or afraid is a very human response to uncertainty or difficult situations. It's okay to acknowledge these feelings. You're safe here to share what's been frightening you.",
            'overwhelmed': "When we feel overwhelmed, it can seem like everything is too much to handle. That's a really difficult place to be. Sometimes breaking things down into smaller pieces can help. What's been feeling like too much lately?"
        }

        # Check for emotion keywords
        for emotion, response in emotion_responses.items():
            if emotion in user_lower:
                return response

        # For other inputs, try to generate with the model but with better prompting
        conversation_context = f"Human: {user_input}\nEmpathetic counselor:"

        # Tokenize
        inputs = self.tokenizer.encode(conversation_context, return_tensors="pt")

        # Generate response
        try:
            with torch.no_grad():
                outputs = self.model.generate(
                    inputs,
                    max_length=len(inputs[0]) + max_length,
                    temperature=temperature,
                    do_sample=True,
                    top_p=0.9,
                    top_k=50,
                    pad_token_id=self.tokenizer.pad_token_id if self.tokenizer.pad_token_id else self.tokenizer.eos_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                    repetition_penalty=1.2,
                    no_repeat_ngram_size=2,
                )

            # Decode response
            full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Extract just the counselor's response
            if "Empathetic counselor:" in full_response:
                response = full_response.split("Empathetic counselor:")[-1].strip()
            else:
                response = ""

            # Clean up and validate response
            response = self.clean_response(response)

            # If response is too short or empty, use fallback
            if len(response.split()) < 5:
                response = self.get_fallback_response(user_input)

        except Exception as e:
            print(f"Generation error: {e}")
            response = self.get_fallback_response(user_input)

        return response

    def get_fallback_response(self, user_input: str) -> str:
        """Generate fallback empathetic responses"""
        fallback_responses = [
            f"I can sense that '{user_input}' is important to you. Can you help me understand more about what you're experiencing?",
            f"Thank you for sharing that with me. It takes courage to open up about what you're going through. How has this been affecting you?",
            f"I hear you saying '{user_input}' and I want you to know that your feelings are completely valid. What would be most helpful for you right now?",
            f"It sounds like you have a lot on your mind. I'm here to listen without judgment. Can you tell me more about what's been weighing on you?",
            f"I appreciate you trusting me with your thoughts. Sometimes it helps just to have someone listen. What else would you like to share?"
        ]

        import random
        return random.choice(fallback_responses)

    def clean_response(self, response: str) -> str:
        """Clean and improve the generated response"""
        # Remove any remaining special tokens
        response = response.replace("<|endoftext|>", "")
        response = response.replace("<|user|>", "")
        response = response.replace("<|assistant|>", "")
        response = response.replace("Human:", "")
        response = response.replace("Empathetic counselor:", "")

        # Remove repetitive patterns
        lines = response.split('\n')
        response = lines[0] if lines else response

        # Ensure response ends properly
        if response and not response.endswith(('.', '!', '?')):
            response += '.'

        # Remove very short or repetitive responses
        if len(response.split()) < 3 or response.count('.') > 3:
            return ""

        return response.strip()

    def chat_session(self):
        """Interactive chat session"""
        print("\n🌟 Mental Health Support Chatbot 🌟")
        print("I'm here to provide emotional support and listen to you.")
        print("Type 'quit', 'exit', or 'bye' to end the conversation.\n")

        while True:
            try:
                user_input = input("\nYou: ").strip()

                if user_input.lower() in ['quit', 'exit', 'bye', 'goodbye']:
                    print("\nChatbot: Take care of yourself. Remember, it's okay to seek help when you need it. 💙")
                    break

                if not user_input:
                    print("Chatbot: I'm here when you're ready to talk.")
                    continue

                # Generate response
                response = self.generate_response(user_input)
                print(f"\nChatbot: {response}")

                # Store conversation
                self.conversation_history.append({
                    "user": user_input,
                    "assistant": response,
                    "timestamp": datetime.now().isoformat()
                })

            except KeyboardInterrupt:
                print("\n\nChatbot: Take care! 💙")
                break
            except Exception as e:
                print(f"\nSorry, I encountered an error: {e}")
                print("Let's try again.")

# ============================================================================
# STREAMLIT WEB INTERFACE
# ============================================================================

def create_streamlit_app():
    """Create Streamlit web interface"""

    st.set_page_config(
        page_title="Mental Health Support Chatbot",
        page_icon="🌟",
        layout="wide"
    )

    st.title("🌟 Mental Health Support Chatbot")
    st.markdown("*A compassionate AI companion for emotional wellness*")

    # Initialize session state
    if "messages" not in st.session_state:
        st.session_state.messages = []
    if "chatbot" not in st.session_state:
        st.session_state.chatbot = None

    # Sidebar for model loading
    with st.sidebar:
        st.header("Model Settings")

        model_option = st.selectbox(
            "Choose model:",
            ["Load trained model", "Use base model (demo)"]
        )

        if st.button("Initialize Chatbot"):
            with st.spinner("Loading model..."):
                try:
                    if model_option == "Load trained model":
                        trainer = MentalHealthChatbotTrainer()
                        if trainer.load_trained_model():
                            st.session_state.chatbot = MentalHealthChatbot(trainer.model, trainer.tokenizer)
                            st.success("Trained model loaded!")
                        else:
                            st.error("Could not load trained model. Using base model instead.")
                            trainer.setup_model_and_tokenizer()
                            st.session_state.chatbot = MentalHealthChatbot(trainer.model, trainer.tokenizer)
                    else:
                        trainer = MentalHealthChatbotTrainer()
                        trainer.setup_model_and_tokenizer()
                        st.session_state.chatbot = MentalHealthChatbot(trainer.model, trainer.tokenizer)
                        st.success("Base model loaded!")
                except Exception as e:
                    st.error(f"Error loading model: {e}")

        st.markdown("---")
        st.markdown("### 💙 Remember")
        st.markdown("- This is for emotional support only")
        st.markdown("- Seek professional help for serious concerns")
        st.markdown("- You're not alone in this journey")

    # Chat interface
    if st.session_state.chatbot:
        # Display chat messages
        for message in st.session_state.messages:
            with st.chat_message(message["role"]):
                st.markdown(message["content"])

        # Chat input
        if prompt := st.chat_input("Share what's on your mind..."):
            # Add user message
            st.session_state.messages.append({"role": "user", "content": prompt})
            with st.chat_message("user"):
                st.markdown(prompt)

            # Generate and display assistant response
            with st.chat_message("assistant"):
                with st.spinner("Thinking..."):
                    response = st.session_state.chatbot.generate_response(prompt)
                    st.markdown(response)

            # Add assistant message
            st.session_state.messages.append({"role": "assistant", "content": response})

    else:
        st.info("Please initialize the chatbot using the sidebar to start chatting.")

        # Demo section
        st.markdown("---")
        st.header("🔧 Training Demo")
        st.markdown("Click below to run a training demonstration:")

        if st.button("Run Training Demo"):
            run_training_demo()

def run_training_demo():
    """Run a complete training demonstration"""
    with st.container():
        st.subheader("Training Progress")

        progress_bar = st.progress(0)
        status_text = st.empty()

        try:
            # Step 1: Initialize data processor
            status_text.text("Step 1/5: Initializing data processor...")
            progress_bar.progress(0.2)
            processor = EmpatheticDataProcessor()

            # Step 2: Load dataset
            status_text.text("Step 2/5: Loading dataset...")
            progress_bar.progress(0.4)
            dataset = processor.load_empathetic_dialogues()

            # Step 3: Initialize trainer
            status_text.text("Step 3/5: Setting up model...")
            progress_bar.progress(0.6)
            trainer = MentalHealthChatbotTrainer()
            trainer.setup_model_and_tokenizer()

            # Step 4: Prepare data
            status_text.text("Step 4/5: Preparing training data...")
            progress_bar.progress(0.8)
            train_data = processor.prepare_training_data(dataset["train"], trainer.tokenizer)
            eval_data = processor.prepare_training_data(dataset["validation"], trainer.tokenizer)

            # Step 5: Train model (shortened for demo)
            status_text.text("Step 5/5: Training model (demo)...")
            progress_bar.progress(1.0)

            # For demo purposes, we'll just show the setup
            st.success("✅ Training setup completed successfully!")
            st.info("In a real scenario, training would take 30-60 minutes depending on your hardware.")

            # Show dataset info
            st.subheader("Dataset Information")
            st.write(f"Training samples: {len(train_data)}")
            st.write(f"Validation samples: {len(eval_data)}")

            # Show sample data
            st.subheader("Sample Training Data")
            sample_text = processor.format_conversation(
                "I'm feeling really anxious about my job interview tomorrow.",
                "I understand how nerve-wracking job interviews can be. It's completely normal to feel anxious about something important like this."
            )
            st.code(sample_text, language="text")

        except Exception as e:
            st.error(f"Error during training demo: {e}")

# ============================================================================
# MAIN EXECUTION FUNCTIONS
# ============================================================================

def main_training():
    """Main function to run the complete training pipeline"""
    print("🌟 Mental Health Support Chatbot - Training Pipeline 🌟\n")

    try:
        # Step 1: Initialize data processor
        print("Step 1: Initializing data processor...")
        processor = EmpatheticDataProcessor()

        # Step 2: Load and prepare dataset
        print("Step 2: Loading EmpatheticDialogues dataset...")
        dataset = processor.load_empathetic_dialogues()
        print(f"Dataset loaded: {len(dataset['train'])} training samples")

        # Step 3: Initialize trainer
        print("Step 3: Setting up model and tokenizer...")
        trainer = MentalHealthChatbotTrainer()
        trainer.setup_model_and_tokenizer()

        # Step 4: Prepare training data
        print("Step 4: Preparing training data...")
        train_data = processor.prepare_training_data(dataset["train"], trainer.tokenizer)
        eval_data = processor.prepare_training_data(dataset["validation"], trainer.tokenizer)

        print(f"Training data prepared: {len(train_data)} samples")

        # Step 5: Train the model
        print("Step 5: Starting model training...")
        trainer.train_model(train_data, eval_data)

        print("✅ Training completed successfully!")

        # Step 6: Test the trained model
        print("Step 6: Testing the trained model...")
        chatbot = MentalHealthChatbot(trainer.model, trainer.tokenizer)

        # Test with sample inputs
        test_inputs = [
            "I'm feeling really anxious today",
            "I can't sleep because I'm worried about work",
            "I feel like nobody understands me"
        ]

        print("\n🔍 Testing responses:")
        for test_input in test_inputs:
            response = chatbot.generate_response(test_input)
            print(f"\nUser: {test_input}")
            print(f"Chatbot: {response}")

        return trainer, chatbot

    except Exception as e:
        print(f"❌ Error during training: {e}")
        return None, None

def main_demo():
    """Run a quick demo with pre-trained model"""
    print("🌟 Mental Health Support Chatbot - Demo Mode 🌟\n")

    try:
        # Initialize with base model
        trainer = MentalHealthChatbotTrainer()
        trainer.setup_model_and_tokenizer()

        # Create chatbot instance
        chatbot = MentalHealthChatbot(trainer.model, trainer.tokenizer)

        # Run interactive session
        chatbot.chat_session()

    except Exception as e:
        print(f"❌ Error during demo: {e}")

# ============================================================================
# GOOGLE COLAB SPECIFIC FUNCTIONS
# ============================================================================

def setup_colab_environment():
    """Setup Google Colab environment"""
    # Check if running in Colab
    try:
        import google.colab
        in_colab = True
    except ImportError:
        in_colab = False

    if in_colab:
        print("🔧 Setting up Google Colab environment...")

        # Install required packages
        os.system("pip install -q transformers datasets torch accelerate streamlit pyngrok")

        # Setup GPU if available
        if torch.cuda.is_available():
            print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
        else:
            print("⚠️ GPU not available, using CPU")

        return True

    return False

def run_streamlit_in_colab():
    """Run Streamlit app in Google Colab using ngrok"""
    try:
        from pyngrok import ngrok
        import subprocess
        import threading

        # Kill any existing ngrok tunnels
        ngrok.kill()

        # Start Streamlit in background
        def run_streamlit():
            subprocess.run(["streamlit", "run", "--server.port", "8501", "--server.address", "0.0.0.0"])

        # Start Streamlit in a separate thread
        streamlit_thread = threading.Thread(target=run_streamlit)
        streamlit_thread.daemon = True
        streamlit_thread.start()

        # Wait a bit for Streamlit to start
        import time
        time.sleep(10)

        # Create ngrok tunnel
        public_url = ngrok.connect(8501)
        print(f"🌐 Streamlit app is running at: {public_url}")
        print("Click the link above to access your Mental Health Support Chatbot!")

        return public_url

    except Exception as e:
        print(f"Error setting up Streamlit in Colab: {e}")
        print("You can still run the command-line version!")
        return None

# ============================================================================
# USAGE EXAMPLES AND DOCUMENTATION
# ============================================================================

def print_usage_instructions():
    """Print comprehensive usage instructions"""
    instructions = """
🌟 MENTAL HEALTH SUPPORT CHATBOT - USAGE INSTRUCTIONS 🌟

This notebook provides a complete implementation of a fine-tuned mental health support chatbot.

📋 QUICK START:
1. Run the installation cell first
2. Choose one of the main functions below:

🔧 TRAINING MODE (Full Pipeline):
```python
# Complete training from scratch
trainer, chatbot = main_training()
```

🎮 DEMO MODE (Quick Test):
```python
# Quick demo with base model
main_demo()
```

🌐 WEB INTERFACE (Streamlit):
```python
# For Google Colab
setup_colab_environment()
run_streamlit_in_colab()

# For local environment
# streamlit run your_script.py
```

📊 CUSTOM TRAINING:
```python
# Initialize components
processor = EmpatheticDataProcessor()
trainer = MentalHealthChatbotTrainer("microsoft/DialoGPT-small")
dataset = processor.load_empathetic_dialogues()

# Setup and train
trainer.setup_model_and_tokenizer()
train_data = processor.prepare_training_data(dataset["train"], trainer.tokenizer)
trainer.train_model(train_data)

# Use the trained model
chatbot = MentalHealthChatbot(trainer.model, trainer.tokenizer)
response = chatbot.generate_response("I'm feeling anxious")
```

🎯 KEY FEATURES:
- Empathetic response generation
- Fine-tuning on EmpatheticDialogues dataset
- Command-line and web interfaces
- GPU acceleration support
- Conversation history tracking
- Emotional tone optimization

⚠️ IMPORTANT NOTES:
- This is for educational/research purposes
- Not a replacement for professional mental health care
- Training may take 30-60 minutes depending on hardware
- Requires ~4GB GPU memory for optimal performance

🔍 TROUBLESHOOTING:
- If dataset loading fails, sample data will be used automatically
- For GPU memory issues, reduce batch size in TrainingArguments
- For Colab issues, try restarting runtime and re-running setup

💡 CUSTOMIZATION:
- Modify EmpatheticDataProcessor for different datasets
- Adjust training parameters in TrainingArguments
- Customize response templates in MentalHealthChatbot
- Add safety filters and content moderation as needed
"""
    print(instructions)

# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Print usage instructions
    print_usage_instructions()

    # Uncomment one of these to run:

    # Full training pipeline
    # trainer, chatbot = main_training()

    # Quick demo
    # main_demo()

    # Streamlit app (uncomment for local use)
    # create_streamlit_app()

    print("\n🌟 Ready to go! Choose your preferred mode above. 🌟")

Using device: cpu

🌟 MENTAL HEALTH SUPPORT CHATBOT - USAGE INSTRUCTIONS 🌟

This notebook provides a complete implementation of a fine-tuned mental health support chatbot.

📋 QUICK START:
1. Run the installation cell first
2. Choose one of the main functions below:

🔧 TRAINING MODE (Full Pipeline):
```python
# Complete training from scratch
trainer, chatbot = main_training()
```

🎮 DEMO MODE (Quick Test):
```python
# Quick demo with base model
main_demo()
```

🌐 WEB INTERFACE (Streamlit):
```python
# For Google Colab
setup_colab_environment()
run_streamlit_in_colab()

# For local environment
# streamlit run your_script.py
```

📊 CUSTOM TRAINING:
```python
# Initialize components
processor = EmpatheticDataProcessor()
trainer = MentalHealthChatbotTrainer("microsoft/DialoGPT-small")
dataset = processor.load_empathetic_dialogues()

# Setup and train
trainer.setup_model_and_tokenizer()
train_data = processor.prepare_training_data(dataset["train"], trainer.tokenizer)
trainer.train_model(train

In [3]:
trainer, chatbot = main_training()

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'empathetic_dialogues' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'empathetic_dialogues' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


🌟 Mental Health Support Chatbot - Training Pipeline 🌟

Step 1: Initializing data processor...
Step 2: Loading EmpatheticDialogues dataset...
Attempting to load EmpatheticDialogues dataset...


README.md: 0.00B [00:00, ?B/s]

empathetic_dialogues.py: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Training data prepared: 16 samples
Step 5: Starting model training...
Starting model training...
Training samples: 16
Validation samples: 5


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss


✅ Training completed successfully!
📁 Model saved to ./mental-health-chatbot
🎉 Training pipeline completed!
✅ Training completed successfully!
Step 6: Testing the trained model...

🔍 Testing responses:

User: I'm feeling really anxious today
Chatbot: Anxiety can be really difficult to deal with. Your feelings are valid, and it takes courage to acknowledge them. What's been causing you to feel anxious lately?

User: I can't sleep because I'm worried about work
Chatbot: Worry can consume so much of our mental energy. It's understandable that you're feeling this way. Sometimes talking through our worries can help make them feel more manageable. What's been on your mind?

User: I feel like nobody understands me
Chatbot: so who are you wanna talk to? The answer is no proble.


In [4]:
main_demo()

🌟 Mental Health Support Chatbot - Demo Mode 🌟

Loading model: microsoft/DialoGPT-small
Model loaded. Vocabulary size: 50260

🌟 Mental Health Support Chatbot 🌟
I'm here to provide emotional support and listen to you.
Type 'quit', 'exit', or 'bye' to end the conversation.


You: hi

Chatbot: Hello! I'm glad you're here. I'm a supportive listener, and I'm here to help you through whatever you're experiencing. How are you feeling today?

You: sad

Chatbot: I can hear that you're feeling sad right now. Sadness can feel overwhelming sometimes. It's completely natural to have these feelings. Would you like to share what's been making you feel this way?

You: me

Chatbot: I'm sure this person could use a little help.

You: bye

Chatbot: Take care of yourself. Remember, it's okay to seek help when you need it. 💙


In [5]:
setup_colab_environment()
url = run_streamlit_in_colab()

🔧 Setting up Google Colab environment...
⚠️ GPU not available, using CPU


ERROR:pyngrok.process.ngrok:t=2025-08-04T13:46:59+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-08-04T13:46:59+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-08-04T13:46:59+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

Error setting up Streamlit in Colab: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.
You can still run the command-line version!
